Кобзарь О.С. Хабибуллин Р.А. 2019 г.

# Модель фонтанирующей скважины

In [ ]:
import sys
sys.path.append('../')

import uniflocpy.uWell.deviation_survey as dev_sev
import uniflocpy.uTools.data_workflow as utool
import uniflocpy.uWell.uPipe as Pipe
import uniflocpy.uWell.Self_flow_well as self_flow_well
import plotly.graph_objs as go
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
import pandas as pd
init_notebook_mode(connected=True)
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt
data = utool.Data()
from uniflocpy.uTools import plotly_workflow
import re

In [ ]:
simple_well = self_flow_well.self_flow_well(h_intake_mes_m=999,h_intake_vert_m=999,
                                            h_bottomhole_mes_m=1000,h_bottomhole_vert_m=1000,
                                            t_earth_init_in_reservoir_c=90, t_bottomhole_c=92,
                                            p_bottomhole_bar=30)
simple_well.well_work_time_sec = 1 

In [ ]:
simple_well.calc_all_from_down_to_up()

In [ ]:
result_df = simple_well.data.get_data_as_df()

In [ ]:
result_df.head()

In [ ]:
result_df.to_excel('test.xlsx')

In [ ]:
result_df = result_df.set_index('well.h_calculated_mes_m')

In [ ]:
result_df.head()

In [ ]:
for i in result_df.columns:
    print(i)

In [ ]:
def find_by_patterns(patterns, list_to_search):
    res = [x for x in list_to_search if re.search(patterns[0], x)] 
    if len(patterns) >1:
        for i in patterns[1:]:
            res = [x for x in res if re.search(i, x)] 
    return res

def plot_specific_columns(result_df, columns_to_plot):
    result_df_to_plot = result_df[columns_to_plot]
    all_traces = plotly_workflow.create_traces_list_for_all_columms(result_df_to_plot, 'lines+markers', swap_xy=True)
    plotly_workflow.plot_func(all_traces, 'Распределение свойств по глубине', 'check.html', reversed_y = True, iplot_option = True)

def filtr_by_antipatterns(init_list, antipatterns):
    new_list = []
    droped_values = []
    for i in init_list:
        for j in antipatterns:
            if j in i:
                droped_values.append(i)
            else:
                new_list.append(i)
    print(f"Удаленные совпадения по антипаттерну: {droped_values}")
    return new_list

def plot_by_patterns(result_df, group_patterns, antipatterns=[]):
    if type(group_patterns[0]) == str:
        columns_to_plot = find_by_patterns(group_patterns, result_df.columns)
    else:
        columns_to_plot = []
        for i in group_patterns:
            this_column_to_plot = find_by_patterns(i, result_df.columns)
            columns_to_plot += this_column_to_plot
    print(f"Найденные совпадения: {columns_to_plot}")
    if len(antipatterns)>0:
        print('cj')
        columns_to_plot = filtr_by_antipatterns(columns_to_plot, antipatterns)
    plot_specific_columns(result_df, columns_to_plot)

In [ ]:
group_patterns = [["p_bar", 'fl'], ['t_c', 'fl'], ['well.t_calculated_earth_init']]
antipatterns = ['cP']
plot_by_patterns(result_df, group_patterns, antipatterns)

In [ ]:
group_patterns = ["rho"]
antipatterns = ['earth']
plot_by_patterns(result_df, group_patterns, antipatterns)

In [ ]:
group_patterns = ["mu"]
antipatterns = ['earth']
plot_by_patterns(result_df, group_patterns, antipatterns)

In [ ]:
simple_well.data.print_all_names()

In [ ]:
def trace(data_x, data_y, namexy):
    tracep = go.Scattergl(
        x = data_x,
        y = data_y,
        name = namexy,
        mode = 'lines'
    )
    return tracep

def plot():
    layout = dict(title = 'Расчет фонтанирующей скважины',  yaxis=dict(title = 'Измеренная глубина, м', autorange='reversed'))

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='basic-scatter')

In [ ]:
#wells = []
#dfs = []
#for i in range(1, 300, 50):
#    simple_well = self_flow_well.self_flow_well(h_intake_mes_m=999,h_intake_vert_m=999,
#                                            h_bottomhole_mes_m=1000,h_bottomhole_vert_m=1000,
#                                            t_earth_init_in_reservoir_c=90, t_bottomhole_c=92,
#                                            d_casing_inner_m=0.12,
#                                            d_tube_inner_m=0.062,
#                                            qliq_on_surface_m3day = 50,
#                                                p_bottomhole_bar=50
#                                       )
#    simple_well.pipe.fluid_flow.fl.rsb_m3m3 = 10 * i
#    simple_well.well_work_time_sec = 1 
#    simple_well.calc_all_from_down_to_up()
#    df = data.extract_data_from_object(simple_well, 'well', [i])
#    dfs.append(df)
#    wells.append(simple_well)

In [ ]:
#traces = []
#for simple_well in wells:
#    trace1 = trace(simple_well.data.get_values(24), simple_well.data.get_values(23),  "Давление, бар")
#    traces.append(trace1)
#data=traces
#plot()    

In [ ]:
trace1 = trace(simple_well.data.get_values(24), simple_well.data.get_values(23),  "Давление, бар")
trace2 = trace(simple_well.data.get_values(25), simple_well.data.get_values(23),  "Температура флюида, С")
trace3 = trace(simple_well.data.get_values(26), simple_well.data.get_values(23),  "Температура земли, С")
data=[trace1,trace2,trace3]
plot()

In [ ]:
trace1 = go.Scatter3d(
    x=simple_well.well_profile.x_displacement_m,
    y=simple_well.well_profile.x_displacement_m * 0,
    z=simple_well.well_profile.h_vert_m * (-1),  # перевод в альтитуду,
    marker=dict(
        size=4,
        colorscale='Viridis',
    ),
    name = 'Построенный профиль',
    )

trace2 = go.Scatter3d(
    x=simple_well.well_profile.get_x_displacement_m(simple_well.well_profile.h_mes_init_data_for_interpolation_m),
    y=simple_well.well_profile.get_x_displacement_m(simple_well.well_profile.h_mes_init_data_for_interpolation_m) * 0,
    z=simple_well.well_profile.h_vert_init_data_for_interpolation_m * (-1),
    mode = 'markers',
    marker = dict(
        size=10
    ),
    name = 'Исходные точки',
    )

data = [trace1, trace2]


layout = dict(title = '3D профиль простой скважины',  yaxis=dict(autorange='reversed'))
fig = dict(data=data, layout=layout)
iplot(fig, filename='simple-3d-scatter')